# Final Project DTSA 5511 Introduction to Deep Learning - Predicting Optimal Fertilizers

This notebook provides a solution for the Predicting Optimal Fertilizers Kaggle competition, developed as a final project for the Deep Learning course at CU Boulder. It includes a problem explanation, exploratory data analysis (EDA), data preprocessing, model building with hyperparameter tuning, results analysis, and a conclusion.

## 1. Deep Learning Problem Description

This project addresses a multiclass classification problem from the Kaggle Playground Series - Season 5, Episode 6. The objective is to predict the optimal 'Fertilizer Name' based on various agricultural features such as temperature, humidity, soil moisture, and nutrient levels (Nitrogen, Phosphorus, Potassium).

**Business Context / Real-World Application:**
Accurate prediction of the most suitable fertilizer type for specific crop and environmental conditions is crucial for optimizing agricultural yield, minimizing waste, reducing environmental impact from over-fertilization, and improving overall farm efficiency. This model can serve as a decision-support tool for farmers, enabling them to make data-driven choices about fertilizer application.

**Dataset Source:**
The dataset is provided by the Kaggle Playground Series, a platform designed to help data scientists practice their skills on real-world-like problems.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## 2. EDA

In [ ]:
# Data Loading
data_dir = '/kaggle/input/playground-series-s5e6/'
try:
    train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))
    sample_submission_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
    print("Data loaded successfully from directory.")
except FileNotFoundError:
    print("Error: Data files not found. Please ensure 'train.csv', 'test.csv', and 'sample_submission.csv' are in the directory.")
    exit()

print("\n--- Train DataFrame Head: ---")
print(train_df.head())
print("\n--- Test DataFrame Head: ---")
print(test_df.head())

# Summaries
print("\n--- Train DataFrame Descriptive Statistics: ---")
print(train_df.describe())
print("\n--- Test DataFrame Descriptive Statistics: ---")
print(test_df.describe())

# Missing Values
print("\n--- Missing values in Train DataFrame: ---")
print(train_df.isnull().sum())
print("\n--- Missing values in Test DataFrame: ---")
print(test_df.isnull().sum())



### Missing Value Handling Strategy:
For this dataset, it appears there are no missing values based on initial checks. If there were, a common strategy for numerical features might be mean/median imputation and for categorical features, mode imputation or a dedicated 'missing' category. Given no missing values, no specific handling is required at this stage.

### Check Feature Distributions

In [ ]:

numerical_features = train_df.select_dtypes(include=np.number).columns.tolist()
# Exclude 'ID' if present and target 'Fertilizer Name'
if 'ID' in numerical_features:
    numerical_features.remove('ID')
if 'id' in numerical_features:  # Corrected column name based on output
    numerical_features.remove('id')


print("\n--- Distributions of Numerical Features: ---")
plt.figure(figsize=(15, 10))
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 4, i + 1)
    sns.histplot(train_df[feature], kde=True)
    plt.title(f'{feature} Distribution')
plt.tight_layout()
plt.show()

### Observation on Skewness:
Features like 'Nitrogen', 'Phosphorus', 'Potassium' might exhibit some skewness. Log transformation or power transformation could be considered if they were highly skewed and a model sensitive to normality was used, but for deep learning, scaling is generally sufficient.


In [ ]:
categorical_features = train_df.select_dtypes(include='object').columns.tolist()
if 'Fertilizer Name' in categorical_features:
    categorical_features.remove('Fertilizer Name') # Target variable

print("\n--- Distributions of Categorical Features (if any - Target variable distribution shown below): ---")

# Target Variable Distribution
plt.figure(figsize=(8, 6))
sns.countplot(y=train_df['Fertilizer Name'], order=train_df['Fertilizer Name'].value_counts().index)
plt.title('Distribution of Target Variable: Fertilizer Name')
plt.show()
print("\nTarget variable counts:")
print(train_df['Fertilizer Name'].value_counts())

### Observation:
The target variable 'Fertilizer Name' appears relatively balanced, which is good for classification.

### Correlation Analysis

In [ ]:
print("\n--- Correlation Matrix of Numerical Features: ---")
correlation_matrix = train_df[numerical_features].corr()
print(correlation_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

### Justification for assumptions:
High correlation between features (e.g., Temperature and Humidity if they were related) could indicate multicollinearity, which might affect traditional linear models but is less of a concern for deep learning models. Correlations with the target (if numerical) would indicate potential importance. Here, we can observe correlations among input features.

### Outlier Detection:
Outliers can be identified visually from box plots or distribution plots, or statistically using methods like IQR rule or Z-scores. For deep learning, scaling methods often mitigate the impact of outliers to some extent. I will rely on StandardScaler to handle the scale differences and implicitly reduce the relative impact of extreme values.

### Data Transformation Hypothesis:
Based on the numerical feature distributions, StandardScaler is hypothesized to be necessary for optimal deep learning model performance. Deep learning models, especially those with gradient-based optimization are sensitive to feature scales. Standardizing features (mean=0, variance=1) helps in faster convergence and prevents features with larger scale from dominating the learning process. No specific log transformation is initially planned as the chosen deep learning models are generally robust to non-normal distributions after scaling.

### Feature Importance Intuition (based on research on Wikipedia):
Intuitively, features like 'Nitrogen', 'Phosphorus', 'Potassium' (N, P, K values) are expected to be highly important as they directly relate to soil fertility and nutrient requirements.
'Crop Type', 'Soil Type', 'Temparature', 'Humidity', and 'Moisture' are also likely to be very relevant as environmental and crop-specific factors directly influence fertilizer needs.
(Domain knowledge from wikipedia)

## 3. Model Building and Training (Deep Learning Focus)

In [ ]:
# Separate features and target
X = train_df.drop(['id', 'Fertilizer Name'], axis=1)
y = train_df['Fertilizer Name']
X_test = test_df.drop('id', axis=1)

# Identify numerical and categorical features
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

### Preprocessing Pipeline
Numerical features will be scaled using StandardScaler. Categorical features will be One-Hot Encoded.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
print(f"Number of target classes: {num_classes}")

# MAP@3 Evaluation Metric (Mean Average Precision at 3)
# Custom metric function as required by the competition
def map_at_3(y_true, y_pred_proba):
    """
    Calculates Mean Average Precision at 3 (MAP@3).
    y_true: True labels (encoded integers).
    y_pred_proba: Predicted probabilities for each class.
    """
    # Get the top 3 predicted class indices for each sample
    # sort in desc order of probability and take top 3 idxs
    top_3_indices = np.argsort(y_pred_proba, axis=1)[:, ::-1][:, :3]

    num_samples = y_true.shape[0]
    avg_precision_sum = 0.0

    for i in range(num_samples):
        true_label = y_true[i]
        predicted_labels_at_k = top_3_indices[i]

        precision_at_k = 0.0
        num_hits = 0
        for k_idx, predicted_label in enumerate(predicted_labels_at_k):
            if predicted_label == true_label:
                num_hits += 1
                # Precision at k: (Number of hits up to k) / k+1
                precision_at_k += num_hits / (k_idx + 1)
                break # Found the true label, no need to check further in top 3
        if num_hits > 0: # Only add if the true label was found in the top 3
            avg_precision_sum += precision_at_k

    return avg_precision_sum / num_samples if num_samples > 0 else 0.0



## Model Selection and Definition

### Model 1: Simple Multi-Layer Perceptron (MLP)

In [ ]:
def create_mlp_model_1(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']) # Accuracy for monitoring during training
    return model



### Model 2: Deeper Multi-Layer Perceptron (MLP)

In [ ]:
def create_mlp_model_2(input_shape, num_classes):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_shape,)),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

### Why these models:
MLPs are a natural choice for tabular data as they can learn complex non-linear relationships between features. They are relatively straightforward to implement and interpret. Model 1 is a basic configuration to establish a baseline. Model 2 is slightly deeper and wider with more dropout to potentially capture more intricate patterns and prevent overfitting. The `softmax` activation is used in the output layer for multiclass classification and `sparse_categorical_crossentropy` is suitable as our target is integer-encoded.

### Training Strategy: Stratified K-Fold Cross-Validation

In [ ]:
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=8, min_lr=0.00001, verbose=1)

models_results = {}
all_test_preds_proba_model1 = []
all_test_preds_proba_model2 = []



## Hyperparameter Optimization (Manual / Experimentation)
For this script, ***hyperparameter tuning*** is demonstrated through varying parameters between model 1 and model 2 (e.g., number of layers, units, dropout rates).
A more exhaustive search could involve Keras Tuner or GridSearchCV with Keras wrappers. Search space for manual tuning (e.g., for `Dense` layers):
- Number of layers: 2 to 4
- Units per layer: 64, 128, 256
- Dropout rates: 0.2, 0.3, 0.4
- Optimizer: 'adam' (chosen for its good default performance)
- Learning Rate: Managed by ReduceLROnPlateau
- Batch size: 32 (a common good starting point)
- Epochs: 100 (with EarlyStopping for efficiency)

In [ ]:
print(f"\nTraining Models with {N_SPLITS}-Fold Stratified Cross-Validation...")

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded)):
    print(f"\n--- Fold {fold+1}/{N_SPLITS} ---")

    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y_encoded[train_idx], y_encoded[val_idx]

    # Apply preprocessing pipeline
    X_train_processed = preprocessor.fit_transform(X_train_fold)
    X_val_processed = preprocessor.transform(X_val_fold)
    # Important: Transform test data using the preprocessor fitted on the *training data of the current fold*
    X_test_processed = preprocessor.transform(X_test) 

    input_shape = X_train_processed.shape[1]

    print("Training Model 1 (Simple MLP)...")
    model1 = create_mlp_model_1(input_shape, num_classes)
    history1 = model1.fit(X_train_processed, y_train_fold,
                          epochs=100,
                          batch_size=32,
                          validation_data=(X_val_processed, y_val_fold),
                          callbacks=[early_stopping, reduce_lr],
                          verbose=0)

    val_loss_model1, val_acc_model1 = model1.evaluate(X_val_processed, y_val_fold, verbose=0)
    print(f"Model 1 - Fold {fold+1} Validation Accuracy: {val_acc_model1:.4f}")

    # Predict probabilities for validation set for MAP@3 calculation
    y_val_pred_proba_model1 = model1.predict(X_val_processed)
    fold_map3_model1 = map_at_3(y_val_fold, y_val_pred_proba_model1)
    print(f"Model 1 - Fold {fold+1} MAP@3: {fold_map3_model1:.4f}")

    # Predict probabilities for test set for ensemble/average prediction later
    test_preds_proba_model1 = model1.predict(X_test_processed)
    all_test_preds_proba_model1.append(test_preds_proba_model1)

    # --- Model 2 Training ---
    print("Training Model 2 (Deeper MLP)...")
    model2 = create_mlp_model_2(input_shape, num_classes)
    history2 = model2.fit(X_train_processed, y_train_fold,
                          epochs=100,
                          batch_size=32,
                          validation_data=(X_val_processed, y_val_fold),
                          callbacks=[early_stopping, reduce_lr],
                          verbose=0)

    val_loss_model2, val_acc_model2 = model2.evaluate(X_val_processed, y_val_fold, verbose=0)
    print(f"Model 2 - Fold {fold+1} Validation Accuracy: {val_acc_model2:.4f}")

    y_val_pred_proba_model2 = model2.predict(X_val_processed)
    fold_map3_model2 = map_at_3(y_val_fold, y_val_pred_proba_model2)
    print(f"Model 2 - Fold {fold+1} MAP@3: {fold_map3_model2:.4f}")

    test_preds_proba_model2 = model2.predict(X_test_processed)
    all_test_preds_proba_model2.append(test_preds_proba_model2)

    if 'Model 1 MAP@3' not in models_results:
        models_results['Model 1 MAP@3'] = []
        models_results['Model 2 MAP@3'] = []
    models_results['Model 1 MAP@3'].append(fold_map3_model1)
    models_results['Model 2 MAP@3'].append(fold_map3_model2)

## 4. Results and Discussion/Conclusion

In [ ]:
print("\n--- Results and Discussion ---")

# Calculate average MAP@3
avg_map3_model1 = np.mean(models_results['Model 1 MAP@3'])
avg_map3_model2 = np.mean(models_results['Model 2 MAP@3'])

print(f"\nAverage MAP@3 for Model 1 (Simple MLP) across {N_SPLITS} folds: {avg_map3_model1:.4f}")
print(f"Average MAP@3 for Model 2 (Deeper MLP) across {N_SPLITS} folds: {avg_map3_model2:.4f}")

# Model Comparison
print("\nModel Comparison:")
print(f"Model 1 MAP@3 scores per fold: {[f'{score:.4f}' for score in models_results['Model 1 MAP@3']]}")
print(f"Model 2 MAP@3 scores per fold: {[f'{score:.4f}' for score in models_results['Model 2 MAP@3']]}")

if avg_map3_model1 > avg_map3_model2:
    print("\nModel 1 (Simple MLP) performed better on average.")
elif avg_map3_model2 > avg_map3_model1:
    print("\nModel 2 (Deeper MLP) performed better on average.")
else:
    print("\nBoth models performed similarly on average.")

### Strengths and Weaknesses:

**Model 1 (Simple MLP):**
- **Strengths:** Simpler, faster to train, less prone to overfitting on smaller datasets.
- **Weaknesses:** May not capture very complex patterns as effectively as deeper models.

**Model 2 (Deeper MLP):**
- **Strengths:** Potentially capable of learning more complex representations, potentially higher accuracy on complex datasets.
- **Weaknesses:** Longer training time, higher risk of overfitting if not properly regularized (e.g., with more dropout).

### Error Analysis:
To perform a detailed error analysis, one would typically examine misclassified samples from the validation set. Key questions would include: Are certain fertilizer types consistently misclassified? Are there patterns in the features of misclassified samples (e.g., specific crop types, extreme environmental conditions)?
Without explicit misclassification examples here, we can infer that errors might arise from:
- Ambiguous feature values where multiple fertilizers are equally plausible.
- Boundary cases between different fertilizer types.
- Insufficient representation of rare fertilizer combinations in the training data.

### Limitations and Future Work:

**Current Limitations:**
- The models are relatively simple MLPs. While effective for tabular data, more advanced architectures could be explored.
- Feature engineering was not extensively performed. More derived features (e.g., ratios, polynomial features) could be beneficial.
- Hyperparameter tuning was manual/experiential. Automated tuning (e.g., Keras Tuner, Optuna) could find better configurations.
- Ensemble methods (e.g., stacking predictions from different folds/models) were not explicitly used for the final prediction process, though averaging test predictions approximates an ensemble.
- No external data was incorporated, which might provide more context for fertilizer recommendations.

**Potential Improvements / Next Steps:**
- **More Complex Architectures:** Explore Residual Networks for tabular data or attention mechanisms.
- **Advanced Feature Engineering:** Create interaction terms, polynomial features or domain-specific features related to plant physiology or soil science.
- **Ensemble Methods:** Implement sophisticated ensembling of multiple diverse models (e.g. XGBoost LightGBM and deep learning models with which I don't have experience yet but heard a lot of positive).
- **Robust Outlier Handling:** Explicitly deal with outliers using robust scaling or outlier removal techniques if they significantly impact performance.
- **Augment Data:** If possible, synthesize new data points, especially for under-represented classes.
- **Explore Other Metrics:** While MAP@3 is primary, other metrcis could provide deeper insights.
- **Interpretability:** Use further advanced techniques to understand feature importance for individual predictions.

**Conclusion:**
This project successfully developed and evaluated Deep Learning models for predicting optimal fertilizer types, achieving competitive MAP@3 scores. The preprocessing pipeline effectively handled numerical scaling and categorical encoding. Stratified K-Fold cross-validation ensured robust evaluation. The results demonstrate the feasibility of using MLPs for this multiclass tabular classification problem. While the current models provide a strong baseline, there's significant scope for improvement through advanced feature engineering, more complex architectures, and comprehensive hyperparameter optimization. The insights gained from this project can contribute to more efficient and sustainable agricultural practices.

## 5. Submission Generation

In [ ]:
# vAverage probabilities from all folds for both models
# This acts as a simple ensemble strategy
ensemble_test_preds_proba = (np.mean(all_test_preds_proba_model1, axis=0) +
                             np.mean(all_test_preds_proba_model2, axis=0)) / 2

# Get top 3 predicts for each sample
top_3_indices = np.argsort(ensemble_test_preds_proba, axis=1)[:, ::-1][:, :3]

# Convert idxs back to original fertilizer names
predicted_fertilizer_names = label_encoder.inverse_transform(top_3_indices.flatten()).reshape(-1, 3)

# Format for submission
submission_predictions = [' '.join(row) for row in predicted_fertilizer_names]

submission_df = pd.DataFrame({'ID': test_df['id'], 'Predictions': submission_predictions}) # Corrected 'Id' to 'ID' for consistency

# Save to CSV
submission_df.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' generated successfully!")
print("\nSample of generated submission file:")
print(submission_df.head())

print("\n--- Script Execution Complete ---")